In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.optim import Adam
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import TensorDataset, DataLoader



In [ ]:
df = pd.read_csv('creditcard.csv')
df.head(1) # give us a sneek preview of the dataset xD


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0


In [ ]:
# Convert X and y to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1) # Add a dimension for BCEWithLogitsLoss

In [ ]:
# Create device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Handle NaN values in y_tensor
nan_mask = torch.isnan(y_tensor)
if torch.any(nan_mask):
    print(f"Found {torch.sum(nan_mask).item()} NaN values in y. Removing corresponding rows.")
    # Remove rows with NaN values in y
    X_tensor = X_tensor[~nan_mask.squeeze()]
    y_tensor = y_tensor[~nan_mask]


In [ ]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42, stratify=y_tensor)



In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.numpy())
X_test_scaled = scaler.transform(X_test.numpy())

X_train_scaled = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_scaled = torch.tensor(X_test_scaled, dtype=torch.float32)


In [ ]:
# Calculate positive class weight for BCEWithLogitsLoss
neg_count = torch.sum(1 - y_train)
pos_count = torch.sum(y_train)
pos_weight = (neg_count / pos_count).item()  # Make it a scalar float


In [ ]:
class FraudDetectionModel(nn.Module):
    def __init__(self, input_dim):
        super(FraudDetectionModel, self).__init__()

        self.fc1 = nn.Linear(input_dim, 64)
        self.dropout1 = nn.Dropout(0.3)

        self.fc2 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(0.3)

        self.output = nn.Linear(32, 1)  # Binary classification

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)

        x = F.relu(self.fc2(x))
        x = self.dropout2(x)

        # Removed sigmoid here to use BCEWithLogitsLoss
        return self.output(x)

    def train_model(self, train_loader, loss_fn, optimizer, num_epochs):
        self.train() # Set the model to training mode

        for epoch in range(num_epochs):
            total_loss = 0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device) # Move data to device
                # Forward pass
                outputs = self(inputs).squeeze(1)  # Shape [B]
                labels = labels.squeeze(1)         # Shape [B]

                # Calculate loss
                loss = loss_fn(outputs, labels)

                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}')

        print("Training complete.")

    def evaluate_model(self, test_loader, threshold):
        self.eval()  # Set the model to evaluation mode
        all_predicted = []
        all_true = []

        with torch.inference_mode():  # Disable gradient calculations
            for inputs, labels in test_loader:
                outputs = self(inputs)
                probabilities = torch.sigmoid(outputs)  # Apply sigmoid to get probabilities
                predicted_classes = (probabilities > threshold).long()  # Convert probabilities to predicted class labels

                all_predicted.append(predicted_classes)
                all_true.append(labels.long()) # Ensure labels are long type for consistency

        # Concatenate all predicted and true labels
        all_predicted = torch.cat(all_predicted).cpu().numpy() # Move to CPU before converting to numpy
        all_true = torch.cat(all_true).cpu().numpy() # Move to CPU before converting to numpy

        # Calculate and print evaluation metrics
        accuracy = accuracy_score(all_true, all_predicted)
        precision = precision_score(all_true, all_predicted)
        recall = recall_score(all_true, all_predicted)
        f1 = f1_score(all_true, all_predicted)

        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")

        return accuracy, precision, recall, f1

In [ ]:
# Instantiate the model
input_dim = X_train_scaled.shape[1]
model = FraudDetectionModel(input_dim).to(device)

# Define the loss function with positive class weight and optimizer
loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight))
optimizer = Adam(model.parameters(), lr=0.001)

print(f"Positive class weight: {pos_weight}")

Positive class weight: 577.2868041992188


## Create data loaders



In [ ]:

train_dataset = TensorDataset(X_train_scaled, y_train)
test_dataset = TensorDataset(X_test_scaled, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("DataLoaders created successfully.")

DataLoaders created successfully.


## Implement training loop



In [ ]:
num_epochs = 10
model.train_model(train_loader, loss_fn, optimizer, num_epochs)

Epoch [1/10], Loss: 0.5105
Epoch [2/10], Loss: 0.2953
Epoch [3/10], Loss: 0.2948
Epoch [4/10], Loss: 0.2702
Epoch [5/10], Loss: 0.2769
Epoch [6/10], Loss: 0.2325
Epoch [7/10], Loss: 0.2195
Epoch [8/10], Loss: 0.2242
Epoch [9/10], Loss: 0.1903
Epoch [10/10], Loss: 0.2363
Training complete.


## Evaluate the model


Evaluate the trained model on the test set by calculating accuracy, precision, recall, and F1-score.



In [ ]:
threshold = 0.99828
accuracy, precision, recall, f1 = model.evaluate_model(test_loader, threshold)

Accuracy: 0.9994
Precision: 0.8144
Recall: 0.8061
F1-score: 0.8103
